In [1]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../src")

from ceres_infer.session import workflow
from ceres_infer.models import model_infer_ens_custom

/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/site-packages/tqdm/std.py:668: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel
Using TensorFlow backend.


In [2]:
import logging
logging.basicConfig(level=logging.INFO)

In [3]:
params = {
    # directories
    'outdir_run': '../out/21.0506 Lx To/L200only_reg_rf_boruta/', # output dir for the run
    'outdir_modtmp': '../out/21.0506 Lx To/L200only_reg_rf_boruta/model_perf/', # intermediate files for each model
    'indir_dmdata_Q3': '../out/21.0506 proc_data to/gene_effect/dm_data_match_to.pkl', # pickled preprocessed DepMap Q3 data
    'indir_dmdata_external': '../out/21.0506 proc_data to/gene_effect/dm_data_to.pkl', # pickled preprocessed DepMap Q3 data
    'indir_genesets': '../data/gene_sets/',
    'indir_landmarks': '../out/19.1013 tight cluster/landmarks_n200_k200.csv', # csv file of landmarks [default: None]

    # notes
    'session_notes': 'L200 landmarks only; regression with random forest-boruta lite iteration',

    # data
    'external_data_name': 'To', # name of external validation dataset
    'opt_scale_data': False, # scale input data True/False
    'opt_scale_data_types': '\[(?:RNA-seq|CN)\]', # data source types to scale; in regexp
    'model_data_source': ['CERES_Lx'],
    'anlyz_set_topN': 10, # for analysis set how many of the top features to look at
    'perm_null': 1000, # number of samples to get build the null distribution, for corr
    'useGene_dependency': False, # whether to use CERES gene dependency (true) or gene effect (false)
    'scope': 'all', # scope for which target genes to run on; list of gene names, or 'all', 'differential'

    # model
    'model_name': 'rf',
    'model_params': {'n_estimators':1000,'max_depth':15,'min_samples_leaf':5,'max_features':'log2'},
    'model_paramsgrid': {},
    'model_pipeline': model_infer_ens_custom,
    'pipeline_params': {'sf_iterThresholds': [], 'sf_topK': None},
    
    # pipeline
    'parallelize': True, # parallelize workflow
    'processes': 24, # number of cpu processes to use
    
    # analysis
    'metric_eval': 'score_test',  # metric in model_results to evaluate, e.g. score_test, score_oob
    'thresholds': {'score_rd10': 0.1,  # score of reduced model - threshold for filtering
                   'recall_rd10': 0.95},  # recall of reduced model - threshold for filtering
    'min_gs_size': 4 # minimum gene set size, to be derived
}

In [ ]:
wf = workflow(params)
pipeline = ['load_processed_data', 'infer']
wf.create_pipe(pipeline)
wf.run_pipe()

INFO:root:Loading preprocessed data...
INFO:root:Adding landmarks...
INFO:root:Running model building and inference...
INFO:root:Total number of processors available: 40
INFO:root:Total number of processors to use: 24
  0%|          | 0/6845 [00:00<?, ?it/s]/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:815: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:815: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "
/storage/home/yur97/anaconda3/envs/cnp/lib/python3.7/site-packages/sklearn/ensemble/_forest.py:815: UserWarning: Some inputs do not have OOB scores. This probably means too few trees wer

In [ ]:
wf = workflow(params)
pipeline = ['load_processed_data', 'load_model_results', 'analyze', 'analyze_filtered', 'derive_genesets']
wf.create_pipe(pipeline)
wf.run_pipe()